## Converting output of NER

We wish to convert the output of NER to a format that can be fed to the RE module.

In [71]:
from pathlib import Path
import shutil
import pprint
import bisect
import re
from collections import defaultdict 
import csv

import scispacy
import spacy

## Function to generate output

In [72]:
#This is just one file. Can be easily generalized

nlp = spacy.load("en_core_sci_sm")

input_path = Path("./data")
output_path = Path("./processed")
 
def generate_output(text_path,ann_path):
    
    with open(text_path, 'r') as f:
        input_lines = f.readlines()

    if len(input_lines)==1:
        doc = nlp(input_lines[0])
        processed_lines = list(doc.sents)
        processed_lines = [s.text for s in processed_lines]
        print(processed_lines[0])

    length_arr = [len(l) for l in processed_lines] 

    length_arr_cum = [0] * len(length_arr)
    temp = 0
    for i in range(len(length_arr)):
        temp += length_arr[i]
        length_arr_cum[i] = temp
#     print(length_arr_cum)             

#     idx = bisect.bisect_right(length_arr_cum,436)
#     print(idx)
    # print(lines[idx][1125-lines_len_cum[idx-1]:1135-lines_len_cum[idx-1]])

    named_entities = [defaultdict(list) for i in range(len(length_arr))]

    with open(ann_path,'r') as f:
        lines = csv.reader(f,delimiter='\t');
        for line in lines:
            info = line[1].split(' ')
            label = info[0]
            start = int(info[1])
            end = int(info[2])
            print(doc[start:end+1],line[2],"wtf")
            idx = bisect.bisect_right(length_arr_cum,start)
            if idx==0:
                baseline = 0
            else:
                baseline = length_arr_cum[idx-1]
            named_entities[idx][label].append((start-baseline,end-baseline,line[0],line[2]))
    
    ## Creating anonymous sentences
    
#     output_sent = []
#     count = 0
#     id_list = []
#     print(processed_lines[0])
#     for i in range(len(length_arr)):

#         for t1 in named_entities[i]['Chemical']:
#             for t2 in named_entities[i]['Gene']:
#                 (start1,end1,id1,w1) = t1
#                 (start2,end2,id2,w2) = t2
#                 print(w1,processed_lines[i][start1:end1],w2,processed_lines[i][start2:end2])
#                 if(start1 < start2):
#                     output_str = processed_lines[i][:start1] + "@CHEMICAL$" + processed_lines[i][end1:start2] + "@GENE$" + processed_lines[i][end2:]
#                     token_tup = (id1,id2)
#                 else:
#                     output_str = processed_lines[i][:start2] + "@GENE$" + processed_lines[i][end2:start1] + "@CHEMICAL$" + processed_lines[i][end1:]
#                     token_tup = (id2,id1)
#                 print(output_str)
#                 output_sent.append(output_str)
#                 id_list.append(token_tup)
    
#     ## Generating input for RE
    
#     name = text_path.stem
#     new_dir = output_path / str(name)
#     new_dir.mkdir(exist_ok= True)
    
#     shutil.copy(text_path, new_dir / text_path.name)
#     shutil.copy(ann_path, new_dir / ("old_" + str(ann_path.name)))
    
# #     with open(new_dir / 'test.tsv','w') as f:
# #         f.write("dummy\tdummy\tsentence\n")
# #         for sent in output_sent:
# #             if '\n' not in sent:
# #                 sent += '\n'
# #             f.write("a\ta\t "+ sent)

#     with open(new_dir / 'pairs.tsv','w') as f:
#         for (t1,t2) in id_list:
#             f.write(t1+"\t"+t2 + "\n")
# # pprint.pprint(named_entities)
                 

In [73]:
## This block is just a test. Comment this out 

text_path = Path("./data/original/sb3000673.txt")
ann_path = Path("./data/original/sb3000673.ann")

generate_output(text_path,ann_path)

  Nonribosomal peptide synthetases (NRPSs) are giant multi-enzymes that carry out sequencial assembly line couplings of amino acids to generate linear or cyclic peptides.
peptide synthetases (NRPSs) are giant multi-enzymes that carry out sequencial assembly line couplings of amino acids to generate linear or cyclic peptides. NRPSs are composed of repeating enzyme domains with Nonribosomal peptide synthetases wtf
organization to activate and couple specific NRPSs wtf
lipopeptide antibiotics daptomycin and A54145E have identical cyclic depsipeptide ring structures and amino acids wtf
These studies on combinatorial biosynthesis have NRPSs wtf
other words, the goal is fatty wtf
. As natural evolution uses mutation amino wtf
as driving forces, a successful synthetic biology process must undoubtedly incorporate and -amino acids wtf
for targeted biological activity (fitness I) and selection for robust fermentation -amino acids wtf
to generate functional peptide coupling devices that can be ex

## Iterating through brat files

Given the root, we will iterate through all the .txt and .ann files that exist inside given path (also iterates through nested subdirectories, thus is recusive)

In [74]:
def walk_dir(root):
    if(root.is_dir()):
        for child in root.iterdir():
            walk_dir(child)
    else:
        if(root.suffix == '.txt'):
            ann_path = root.with_suffix(".ann")
            if(ann_path.exists()):
                generate_output(root,ann_path)

In [47]:
walk_dir(input_path)